In [3]:
from torchtext import data, datasets
from torchtext.vocab import GloVe, FastText, CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

In [4]:
if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [5]:
is_cuda = torch.cuda.is_available()

In [6]:
TEXT = data.Field(lower=True, fix_length=200, batch_first=True)
LABEL = data.Field(sequential=False,)

In [7]:
train, test = IMDB.splits(TEXT, LABEL)

In [8]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300), max_size=10000, min_freq=10)
LABEL.build_vocab(train,)

In [9]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [10]:
batch = next(iter(train_iter))

In [11]:
n_vocab = len(TEXT.vocab)
hidden_size = 100

In [18]:
class IMDBCnn(nn.Module):
    
    def __init__(self, vocab, hidden_size, n_cat, bs=1, kernel_size=3, max_len=200):
        super().__init__()
        self.hidden_size = hidden_size
        self.bs = bs
        self.e = nn.Embedding(n_vocab, hidden_size)
        self.cnn = nn.Conv1d(max_len, hidden_size, kernel_size)
        self.avg = nn.AdaptiveAvgPool1d(10)
        self.fc = nn.Linear(1000, n_cat)
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self, inp):
        bs = inp.size()[0]
        if bs != self.bs:
            self.bs = bs
        e_out = self.e(inp)
        cnn_o = self.cnn(e_out)
        cnn_avg = self.avg(cnn_o)
        cnn_avg = cnn_avg.view(self.bs,-1)
        fc = F.dropout(self.fc(cnn_avg), p=0.5)
        return self.softmax(fc)

In [19]:
model = IMDBCnn(n_vocab, hidden_size, n_cat=3, bs=32, kernel_size=2)
model = model.cuda()

optimizer = optim.Adam(model.parameters(), lr=1e-3)

def fit(epoch, model, data_loader, phase='training', volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile = True
    
    running_loss = 0.0
    running_correct = 0
    
    for batch_idx, batch in enumerate(data_loader):
        text, target = batch.text, batch.label
        
        if is_cuda:
            text, target = text.cuda(), target.cuda()
        
        if phase=='training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output, target)
        
        running_loss += F.nll_loss(output, target, size_average=False).data
        preds = output.data.max(dim=1, keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = running_loss / len(data_loader.dataset)
    accuracy = 100. * running_correct.item() / len(data_loader.dataset)
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    
    return loss,accuracy

In [20]:
%%time

train_losses, train_accuracy = [], []
val_losses, val_accuracy = [], []

for epoch in range(1,5):
    
    epoch_loss, epoch_accuracy = fit(epoch, model, train_iter, phase='training')
    val_epoch_loss, val_epoch_accuracy = fit(epoch, model, test_iter, phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

C:\Users\Joshua\Anaconda3\envs\pytorch\lib\site-packages\torch\nn\_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is  0.84 and training accuracy is 11811/25000     47.24
validation loss is  0.75 and validation accuracy is 13324/25000      53.3
training loss is  0.67 and training accuracy is 14180/25000     56.72
validation loss is  0.69 and validation accuracy is 14173/25000     56.69
training loss is  0.59 and training accuracy is 15112/25000     60.45
validation loss is  0.66 and validation accuracy is 14500/25000      58.0
training loss is  0.55 and training accuracy is 15477/25000     61.91
validation loss is   0.7 and validation accuracy is 14228/25000     56.91
Wall time: 46.5 s
